<table style="font-size: 1em; padding: 0; margin: 0;">

<tr style="vertical-align: top; padding: 0; margin: 0;background-color: #ffffff">
        <td style="vertical-align: top; padding: 0; margin: 0; padding-right: 15px;">
    <p style="background: #182AEB; color:#ffffff; text-align:justify; padding: 10px 25px;">
        <strong style="font-size: 1.0em;"><span style="font-size: 1.2em;"><span style="color: #ffffff;">Deep Learning </span> for Satellite Image Classification (Manning Publications)</span><br/>by <em>Daniel Buscombe</em></strong><br/><br/>
        <strong>> Chapter 4: Model Training and Evaluation </strong><br/>
    </p>           
        
<p style="border: 1px solid #182AEB; border-left: 15px solid #182AEB; padding: 10px; text-align:justify;">
    <strong style="color: #182AEB">What you learned in Part 3.</strong>  
    <br/>In Part 3, you learned how to set up and merge image generators, carry out data augmentation, put together and train deep learning models for denoising imagery using keras/Tensorflow 2, and put together a trainable UNet model implementation for semantic segmentation.
    </p>
    
<p style="border: 1px solid #ff5733; border-left: 15px solid #ff5733; padding: 10px; text-align:justify;">
    <strong style="color: #ff5733">What you will learn in this Part.</strong>  
    <br/>In Part 4, you will first learn a few core concepts in training deep learning models, create checkpoints and callback functions for training your UNet model, and then train that model on a non-augmented and augmented Sentinel2-cloudless dataset. Finally, you will plot the training histories of the model training and validation, and use the model on a test image for prediction.
    </p>    

Switch to (or install) tensorflow 2 (if you are on colab)

In [ ]:
colab = 0
#colab = 1

if colab==1:
    %tensorflow_version 2.x
    #!pip install --default-timeout=1000 tensorflow-gpu==2.0   

You may have to restart the runtime and/or change runtime type here, if the following doesn't show Tensorflow version 2, and a GPU available

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
tf.test.is_gpu_available()

<table style="font-size: 1em; padding: 0; margin: 0;">

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #182AEB;">
    </small><br/>Key concepts in  <br/> training deep learning models</h1>
<br/>
<p style="border-left: 15px solid #182AEB; text-align:justify; padding: 0 10px;">
A a high level, model training consists of, upon each successive iteration:

* Evaluate the current model but computing its _loss_, which is the mismatch between the a batch of validation labels and the model's current estimate of those labels

* Adjust the weights of the network (the parameters of the model) to try to reduce the loss

* Repeat the above until the model solution converges (it cannot be improved)
</p>
        </tr>
        </table>

### Backpropagation
Backpropagation is a method to update the weights in the neural network by taking into account the actual output and the desired output. 

### Updating weights
In a neural network, weights are updated as follows:

* Step 1: Take a batch of training data and perform forward propagation to compute the loss. 
* Step 2: Backpropagate the loss to get the gradient of the loss with respect to each weight. 
* Step 3: Use the gradients to update the weights of the network.

![](https://stanford.edu/~shervine/images/update-weights.png)
[source](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-deep-learning-tips-and-tricks)

### Optimizing convergence

Stochastic Gradient Descent is a popular way to find how to minimize a cost function  (called finding a global minima)

For each example in the data:

* find the value predicted by the neural network 
* calculate the loss from the loss function 
* find partial derivatives of the loss function, these partial derivatives produce gradients
* use the gradients to update the values of weights and biases

A more detailed yet accessible explanation may be found [here](http://ruder.io/optimizing-gradient-descent/) 

#### Learning rate
The learning rate, indicates at which pace the weights get updated. It can be fixed or adaptively changed. 

* If the learning rate is low, then training is more reliable, but optimization will take a lot of time because steps towards the minimum of the loss function are tiny.

* If the learning rate is high, then training may not converge or even diverge. Weight changes can be so big that the optimizer overshoots the minimum and makes the loss worse.

![](https://cdn-images-1.medium.com/max/1000/0*QwE8M4MupSdqA3M4.png)

[source](https://towardsdatascience.com/a-look-at-gradient-descent-and-rmsprop-optimizers-f77d483ef08b)


#### Adaptive learning rates
Letting the learning rate vary when training a model can reduce the training time and improve the numerical optimal solution. 

* The Adam optimizer is the most commonly used technique. 

* Here we are using a technique called Root Mean Squared Propagation" or RMSprop an adaptive version of Stochastic Gradient Descent. A more detailed yet accessible explanation may be found [here](http://ruder.io/optimizing-gradient-descent/) 


#### Loss function

Binary cross-entropy compares the log of probabilities of each predicted class with the log of the observed values.  

### Semantic segmentation of water using U-Net

In [ ]:
# from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
#imagery
file_id = '1iMfIjr_ul49Ghs2ewazjCt8HMPfhY47h'
destination = 's2cloudless_imagery.zip'
if colab==1:
    download_file_from_google_drive(file_id, destination)
    
#labels
file_id = '1c7MpwKVejoUuW9F2UaF_vps8Vq2RZRfR'
destination = 's2cloudless_label_imagery.zip'
if colab==1:
    download_file_from_google_drive(file_id, destination)    

In [ ]:
import zipfile
def unzip(f):
    """
    f = file to be unzipped
    """    
    with zipfile.ZipFile(f, 'r') as zip_ref:
        zip_ref.extractall()
        
if colab==1:
    unzip('s2cloudless_imagery.zip')
    unzip('s2cloudless_label_imagery.zip')        

#### Training a Unet to detect water in Sentinel2-cloudless imagery

Import the libraries we will need:

In [ ]:
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import Concatenate, Conv2DTranspose
from tensorflow.keras.models import Model
import numpy as np
import json, os
from random import shuffle
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

Use the same IOU and Unet functions we used in the last Part

In [ ]:
def mean_iou(y_true, y_pred):
    yt0 = y_true[:,:,:,0]
    yp0 = tf.keras.backend.cast(y_pred[:,:,:,0] > 0.5, 'float32')
    inter = tf.math.count_nonzero(tf.logical_and(tf.equal(yt0, 1), tf.equal(yp0, 1)))
    union = tf.math.count_nonzero(tf.add(yt0, yp0))
    iou = tf.where(tf.equal(union, 0), 1., tf.cast(inter/union, 'float32'))
    return iou

In [ ]:
def unet(sz = (512, 512, 3)):
  inputs = Input(sz)
  _ = inputs
  
  #down sampling 
  f = 8
  layers = []
  
  for i in range(0, 6):
    _ = Conv2D(f, 3, activation='relu', padding='same') (_)
    _ = Conv2D(f, 3, activation='relu', padding='same') (_)
    layers.append(_)
    _ = MaxPooling2D() (_)
    f = f*2
  ff2 = 64 
  
  #bottleneck 
  j = len(layers) - 1
  _ = Conv2D(f, 3, activation='relu', padding='same') (_)
  _ = Conv2D(f, 3, activation='relu', padding='same') (_)
  _ = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (_)
  _ = Concatenate(axis=3)([_, layers[j]])
  j = j -1 
  
  #upsampling 
  for i in range(0, 5):
    ff2 = ff2//2
    f = f // 2 
    _ = Conv2D(f, 3, activation='relu', padding='same') (_)
    _ = Conv2D(f, 3, activation='relu', padding='same') (_)
    _ = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (_)
    _ = Concatenate(axis=3)([_, layers[j]])
    j = j -1 
    
  #classification 
  _ = Conv2D(f, 3, activation='relu', padding='same') (_)
  _ = Conv2D(f, 3, activation='relu', padding='same') (_)
  outputs = Conv2D(1, 1, activation='sigmoid') (_)
  
  #model creation 
  model = Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = [mean_iou])
  
  return model  

In [ ]:
model = unet()
model.summary()

#### Checkpoints

Checkpoints are a way to save the current state of your model so that you can pick up from where you left off. They capture

* The architecture of the model, allowing you to re-create the model

* The weights of the model (for now, these will essentially be random numbers)

* The training configuration (loss, optimizer, epochs, and other meta-information - I'll explain what all this means later)

With Colab, GPU kernels may terminate, suddenly and without warning. Therefore, it is often a good idea to keep number of training epochs small, but to use checkpoints

Then, it looks from images and corresponding label images, and trains a deep learning model **end-to-end**. This means that all the parameters in the models will be learned from our data, using a single large network that automatically learns how to map inputs (pixels and regions of images) to outputs (our classes). This is computationally intensive, hence the need for GPUs. Our workflow would therefore be described in the DL literature as 'end-to-end'. 

We will are not using 'transfer learning', which is the (usually less computationally intensive) process of using not only existing models, but also re-purposing existing parameters sets learned from other data.

Functions to save the model at each epoch and show some predictions 

In [ ]:
# inheritance for training process plot 
class PlotLearning(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        #self.fig = plt.figure()
        self.logs = []
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('mean_iou'))
        self.val_acc.append(logs.get('val_mean_iou'))
        self.i += 1
        print('i=',self.i,'loss=',logs.get('loss'),'val_loss=',logs.get('val_loss'),'mean_iou=',logs.get('mean_iou'),'val_mean_iou=',logs.get('val_mean_iou'))
        
        #choose a random test image and preprocess
        path = np.random.choice(test_files)
        infile = f's2cloudless_imagery/data/{path}'
        raw = Image.open(infile)
        raw = np.array(raw.resize((512, 512)))/255.
        raw = raw[:,:,0:3]
        
        #predict the mask 
        pred = 255*model.predict(np.expand_dims(raw, 0)).squeeze()
        print(np.max(pred))
                
        #mask post-processing 
        msk  = (pred>60).astype('int') #100       
        msk = np.stack((msk,)*3, axis=-1)
        #msk[msk >= 0.5] = 1 
        #msk[msk < 0.5] = 0 
        
        #show the mask and the segmented image 
        combined = np.concatenate([raw, msk, raw* msk], axis = 1)
        plt.axis('off')
        plt.imshow(combined)
        plt.show()

In [ ]:
def build_callbacks():
        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='unet.h5', verbose=0, save_best_only=True, save_weights_only=True)
        callbacks = [checkpointer, PlotLearning()]
        return callbacks

Colab users, you will need to quickly upload your copy of ```all_labels.json``` here. The json file contains the image names

In [ ]:
# load the labels than contain the image list
data = json.load(open('all_labels.json'))
images = sorted(data.keys())

Use the image batch generator that we used in the previous Part

In [ ]:
def image_batch_generator(files, batch_size = 32, sz = (512, 512)):
  
  while True: # this is here because it will be called repeatedly by the training function
    
    #extract a random subset of files of length "batch_size"
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs (x) and outputs (y)
    batch_x = []
    batch_y = []
    
    #cycle through each image in the batch
    for f in batch:

        #preprocess the raw images 
        rawfile = f's2cloudless_imagery/data/{f}'
        raw = Image.open(rawfile)
        raw = raw.resize(sz)
        raw = np.array(raw)

        #check the number of channels because some of the images are RGBA or GRAY
        if len(raw.shape) == 2:
            raw = np.stack((raw,)*3, axis=-1)

        else:
            raw = raw[:,:,0:3]
            
        #get the image dimensions, find the min dimension, then square the image off    
        nx, ny, nz = np.shape(raw)
        n = np.minimum(nx,ny)
        raw = raw[:n,:n,:] 
            
        batch_x.append(raw)
        
        #get the masks. 
        maskfile = rawfile.replace('s2cloudless_imagery','s2cloudless_label_imagery')+'_mask.jpg'
        mask = Image.open(maskfile)
        # the mask is 3-dimensional so get the max in each channel to flatten to 2D
        mask = np.max(np.array(mask.resize(sz)),axis=2)
        # water pixels are always greater than 100
        mask = (mask>100).astype('int')
        
        mask = mask[:n,:n]

        batch_y.append(mask)

    #preprocess a batch of images and masks 
    batch_x = np.array(batch_x)/255. #divide image by 255 to normalize
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,3) #add singleton dimension to batch_y

    yield (batch_x, batch_y) #yield both the image and the label together

Define our one hyperparameter (batch size), the proportion of the dataset to use for training (60%), then get all the files, shuffle them randomly, draw 60% for training, then use the rest for testing

In [ ]:
batch_size = 8

prop_train = 0.6

all_files = os.listdir('s2cloudless_imagery/data')
shuffle(all_files)

split = int(prop_train * len(all_files))

#split into training and testing
train_files = all_files[0:split]
test_files  = all_files[split:]

Based on these files, define train and test generators

In [ ]:
train_generator = image_batch_generator(train_files, batch_size = batch_size)
test_generator  = image_batch_generator(test_files, batch_size = batch_size)

We're going to define the number of steps per epoch (based on the batch size) and also the number of epochs

In [ ]:
train_steps = len(train_files) //batch_size
test_steps = len(test_files) //batch_size

print(train_steps)
print(test_steps)

Start model training. Use `verbose = 0` because we have a callback function that displays a test image and prediction at the end of each epoch. We `use_multiprocessing = True` in case keras complains about our image generator function not being thread-safe 

In [ ]:
history = model.fit_generator(train_generator, 
                    epochs = 100, steps_per_epoch = train_steps,
                    validation_data = test_generator, validation_steps = test_steps,
                    callbacks = build_callbacks(), verbose = 0,
                    use_multiprocessing=True)

#### Training with image augmentation

In the previous Part we saw how to use the ```ImageDataGenerator``` class in ```keras```, which we use below to generate batches of 1 images randomly rotated up to 45 degrees, flipped horizontally, and/or zoomed in/out up to 20%

In [ ]:
# generate batches of one because the generator will be used
# to print imagery to file

batch_size=1

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,
        featurewise_std_normalization=False,     
        shear_range=0,
        zoom_range=0.2,
        rotation_range=45,
        horizontal_flip=True)

img_generator = train_datagen.flow_from_directory(    
        's2cloudless_imagery',
        target_size=(2048, 2048), #opt for a fairly small image size for memory efficiency
        batch_size=batch_size,
        class_mode=None, seed=111, shuffle=False)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,
        featurewise_std_normalization=False,     
        shear_range=0,
        zoom_range=0.2,
        rotation_range=45,
        horizontal_flip=True)

mask_generator = test_datagen.flow_from_directory(
        's2cloudless_label_imagery',
        target_size=(2048, 2048),
        batch_size=batch_size,
        class_mode=None, seed=111, shuffle=False)

Convert each image and label image to floating point between 0 and 1 while combing the two generators, then generate and save to file 100 augmented images and associated labels

In [ ]:
n_aug_files = 100

train_generator2 = (tuple(np.array(pair, dtype='float64')/255) for pair in zip(img_generator, mask_generator))

counter = 0
while counter<n_aug_files:
    x, y = next(train_generator2)
    matplotlib.image.imsave('s2cloudless_label_imagery'+os.sep+'data'+os.sep+"augimage00"+str(counter)+".jpg_mask.jpg", np.squeeze(y[0])) 
    matplotlib.image.imsave('s2cloudless_imagery'+os.sep+'data'+os.sep+"augimage00"+str(counter)+".jpg", np.squeeze(x[0]))    
    counter += 1

While the above is running, you can check in the file folders to see the new augmented imagery

(in case you want to undo the above, uncomment and run the cell below)

In [ ]:
#! rm s2cloudless_label_imagery/data/aug*.jpg
#! rm s2cloudless_imagery/data/aug*.jpg

Shuffle files again to pick a new set of training and testing files, and refine the test and train generators for the next round of tests

In [ ]:
#batch size can now be higher because you have a function 
#generating augmented imagery
batch_size = 16

shuffle(all_files)

split = int(prop_train * len(all_files))

#split into training and testing
train_files = all_files[0:split]
test_files  = all_files[split:]

train_steps = len(train_files) //batch_size
test_steps = len(test_files) //batch_size

print('Steps per training epoch: %i' % (train_steps))
print('Steps per testing epoch: %i' % (test_steps))

train_generator = image_batch_generator(train_files, batch_size = batch_size)
test_generator  = image_batch_generator(test_files, batch_size = batch_size)

Remove the previous model weights file and run the model training function again, this time using augmented imagery, and double the training epochs (200) to capitalize on the model having more training data

In [ ]:
os.remove('unet.h5')

By using a checkpoint weights file, you could use smaller numbers of epochs and repeatedly run the cell below. However, that will meann it is harder to keep track of training history

In [ ]:
history_aug = model.fit_generator(train_generator, 
                    epochs = 100, steps_per_epoch = train_steps,
                    validation_data = test_generator, validation_steps = test_steps,
                    callbacks = build_callbacks(), verbose = 0,
                    use_multiprocessing=True)

<table style="font-size: 1em; padding: 0; margin: 0;">

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #182AEB;">
    </small><br/>Evaluating <br/> semantic segmentation results</h1>
<br/>
<p style="border-left: 15px solid #182AEB; text-align:justify; padding: 0 10px;">
We evaluate how good a model is at prediction using an unseen test set of images and an evaluation metric. 
</p>
        </tr>
        </table>

#### Examine training history

`history` contains the training and validation metrics that we can then plot as a function of epoch

`history_aug` is the equivalent for the second set of training using augmented data. 

We have both training and validation losses and mean IoU scores:

In [ ]:
print(history.history.keys())

In [ ]:
print(history_aug.history.keys())

The plot below shows the training history of the original non-augmented and augmented training runs. You can see by comparing the four curves that augmentation doesn't helps a little with overall final training accuracy, which seems to start to plateau at around 0.85 for both augmented models. Augmentation also seemsto help the model generalize better; test set scores are generally higher at around 0.7

In [ ]:
# summarize history for iou
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.plot(history.history['mean_iou'],'k',lw=1) #plot the training iou curve
plt.plot(history.history['val_mean_iou'],'r',lw=1) #plot the testing iou curve
plt.ylim(0,1) #put limits of y axis
plt.axhline(y=0.85) #draw a horizontal line at 85% IoU
plt.title('Model IoU') #add title, labels and a legend
plt.ylabel('IoU')
plt.xlabel('Epoch number')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(122)
plt.plot(history_aug.history['mean_iou'],'k',lw=1)
plt.plot(history_aug.history['val_mean_iou'],'r',lw=1)
plt.title('Augmented model IoU')
plt.ylim(0,1)
plt.axhline(y=0.85)
plt.ylabel('IoU')
plt.xlabel('Epoch number')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Augmentation improves IoU too, approaching the 5% error line

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.plot(history.history['loss'],'k',lw=1)
plt.plot(history.history['val_loss'],'r',lw=1)
plt.ylim(0,1)
plt.axhline(y=0.05)
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch number')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(122)
plt.plot(history_aug.history['loss'],'k',lw=1)
plt.plot(history_aug.history['val_loss'],'r',lw=1)
plt.title('Augmented model loss')
plt.ylim(0,1)
plt.axhline(y=0.05)
plt.ylabel('Loss')
plt.xlabel('Epoch number')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#### Testing

In [ ]:
# open an image 
raw = Image.open('s2cloudless_imagery'+os.sep+'data'+os.sep+test_files[0])

# resize and rescale, which is what the model is expecting as inputs
raw = np.array(raw.resize((512, 512)))/255.
##raw = raw[:,:,0:3]

#predict the mask 
pred = model.predict(np.expand_dims(raw, 0))

#mask post-processing 
msk  = pred.squeeze()
msk = np.stack((msk,)*3, axis=-1)
# binarize
msk[msk >= 0.5] = 1 
msk[msk < 0.5] = 0 

#show the mask and the segmented image 
combined = np.concatenate([raw, msk, raw* msk], axis = 1)
plt.axis('off')
plt.imshow(combined)
plt.show()

What next?

It did _reasonably_ well on the training data but not so well. We probably need more data. This is where the NWPU-RESISC45 imagery might come to the rescue!

![](https://github.com/dbuscombe-usgs/cdi_dl_workshop/raw/67e8c84d0e0b89b024814ef2e8f3bed091ee0c4e/Day2/figs/Picture1.png)

<table style="font-size: 1em; padding: 0; margin: 0;">
<p style="border: 1px solid #ff5733; border-left: 15px solid #ff5733; padding: 10px; text-align:justify;">
    <strong style="color: #ff5733">Deliverable</strong>  
    <br/>The deliverable for Part 4 is a jupyter notebook showing a workflow to train and test U-Net models for training using augmented NWPU-RESISC45 lake images and corresponding labels, and using augmented sentinel-2 cloudless imagery and corresponding labels. So, there are two U-Net based models in total. You should compare their outputs. Finally, you should test the model trained on NWPU imagery on the s2cloudless imagery.  This will mostly test your understanding of the generic yet complex workflow of training, testing and comparing models based on a test data set, as well as how to interpret evaluation metrics. The baseline evaluation of testing the model trained on NWPU imagery on the s2cloudless imagery (in machine learning called "transfer learning") is what you will optimize in Part 5.
    </p>

<table style="font-size: 1em; padding: 0; margin: 0;">

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #182AEB;">
    </small><br/>Going further: <br/> train models with your own data</h1>
<br/>
<p style="border-left: 15px solid #182AEB; text-align:justify; padding: 0 10px;">
If you developed your own imagery in Part 2, you should be able to adapt the workflow presented here to developing training and testing sets (augmented or not). You could choose to build separate UNet models for each dataset, or merge datasets together to build one 'master' UNet model. Most deep learning studies report that model performance scale with data set size.
</p>
<p style="border-left: 15px solid #6019D6; padding: 0 10px; text-align:justify;">
    <strong style="color: #6019D6;">Tip.</strong> 
Many deep learning studies report that model performance scale with model size, so you could add filters to the unet model atchitecture presented in this part
</p>
        </tr>
        </table>